This notebook showcases usage of openAI's gpt API to evaluate open-ended questions.

In [1]:
import configparser
import openai
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['DEFAULT']['OPENAI_API_KEY']
GPT_MODEL = config['DEFAULT']['GPT_MODEL']

client = openai.OpenAI(api_key=API_KEY)

In [15]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL,
                            max_tokens=150, temperature=0.7, top_p=1.0, frequency_penalty=0.0, 
                            presence_penalty=0.0):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [16]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [26]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of California?"},
]

response = chat_completion_request(
    messages=messages
)

messages.append({"role": "assistant", "content": response.choices[0].message.content})

pretty_print_conversation(messages)

system: You are a helpful assistant.

user: What is the capital of California?

assistant: The capital of California is Sacramento.



How to enforce the model to answer the question in a specific way?

- We can provide a prompt that guides the model to answer the question in a specific way.
- We can also provide a one-shot example that shows the model how to answer the question.
- We can also perform function calls to enforce the model to answer the question in a specific way.